In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
from torchvision import models
from torchvision import transforms
import os

In [3]:
if(os.path.split(os.getcwd())[1] == "Transfer_Learning" or os.path.split(os.getcwd())[1] == "semi-supervised" or os.path.split(os.getcwd())[1] == "self-supervised"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

Current Working Directory: progettoVIPM


In [4]:
cuda=True

In [6]:
from utils.loadersAndEnums import datasets, ImageDataset
from torch.utils.data import DataLoader, ConcatDataset, random_split



trainM = ImageDataset(dataset=datasets.TRAINING_MIXED,network_input_size=256,cuda = cuda)
trainL = ImageDataset(dataset=datasets.TRAINING_LABELED_80,network_input_size=256,cuda=cuda)
validation = ImageDataset(dataset=datasets.VALIDATION_LABELED_20,network_input_size=256,cuda=cuda)

In [6]:
for m in trainM:
    print(m)

(tensor([[[0.7527, 0.6741, 0.6592,  ..., 0.5740, 0.5626, 0.5697],
         [0.7451, 0.6312, 0.6186,  ..., 0.4959, 0.4844, 0.5105],
         [0.6690, 0.5646, 0.6011,  ..., 0.4443, 0.4372, 0.4755],
         ...,
         [0.7076, 0.5896, 0.4007,  ..., 0.6161, 0.6864, 0.7070],
         [0.7778, 0.6106, 0.4429,  ..., 0.5774, 0.6712, 0.7188],
         [0.8152, 0.6499, 0.6335,  ..., 0.5918, 0.7170, 0.7722]],

        [[0.8257, 0.7614, 0.7667,  ..., 0.7186, 0.7161, 0.7287],
         [0.8261, 0.7236, 0.7267,  ..., 0.6478, 0.6460, 0.6713],
         [0.7657, 0.6727, 0.7248,  ..., 0.6199, 0.6142, 0.6399],
         ...,
         [0.7392, 0.6387, 0.4776,  ..., 0.6715, 0.7317, 0.7500],
         [0.7995, 0.6521, 0.5197,  ..., 0.6333, 0.7125, 0.7518],
         [0.8369, 0.6922, 0.7035,  ..., 0.6474, 0.7587, 0.7962]],

        [[0.7700, 0.6998, 0.7015,  ..., 0.8324, 0.8297, 0.8455],
         [0.7751, 0.6669, 0.6675,  ..., 0.7624, 0.7551, 0.7871],
         [0.7127, 0.6171, 0.6656,  ..., 0.7339, 0.7278, 0

KeyboardInterrupt: 

In [7]:
trainM = DataLoader(trainM,128,shuffle=False)
trainL = DataLoader(trainL,128,shuffle=False)
validation = DataLoader(validation, shuffle=False)

In [8]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  # (B, 64, H/2, W/2)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (B, 64, H/4, W/4)
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1), # (B, 128, H/8, W/8)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (B, 128, H/16, W/16)
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1), # (B, 256, H/32, W/32)
            nn.ReLU()
        )
    def forward(self, x):
        return self.encoder(x)


class ExtendedEncoder(nn.Module):
    def __init__(self, base_encoder):
        super(ExtendedEncoder, self).__init__()
        self.base_encoder = base_encoder

        # Dynamically compute flatten_dim
        dummy_input = torch.randn(1, 3, 256, 256).to(device)  # Simulate input
        dummy_output = self.base_encoder(dummy_input)  # Get encoder output
        self.flatten_dim = dummy_output.view(1, -1).size(1)  # Flatten dimension
        
        # Fully connected layer
        self.fc = nn.Linear(self.flatten_dim, 251)
        self.softmax = nn.Softmax(dim=1)
        
        for param in self.base_encoder.parameters():
            param.requires_grad = False


    def forward(self, x):
        x = self.base_encoder(x)  # Base encoder
        x = x.view(x.size(0),-1)  # Flatte
        x = self.fc(x)  # Fully connected layer
        x = self.softmax(x)
        return x

In [9]:
PATH = "self-supervised\models\AutoEncoderModelV2.pth"

model = torch.load(PATH)

In [10]:
encoder = model.base_encoder
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder.to(device)

Encoder(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
  )
)

In [14]:
for inp,lab in trainM:
    print(lab.numpy())

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6]
[ 6  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12 12]
[12 12 12 12 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 16 16 16 16 16 16 16 16
 16 16 16 16 16 16 16 16 16 16 16 16 17 17 17 17 17 17 17 17 17 17 17 17
 17 17 17 17 17 17 17 17 18 18 18 18 18 18 18 18 18 18 1

KeyboardInterrupt: 

In [15]:
feature_train = []
labels_train = []
encoder.eval()
with torch.no_grad():
    for inputs,labels in trainL:
        inputs = inputs.to(device)
        outputs = encoder(inputs)
        outputs = outputs.view(outputs.size(0),-1)  # Flatte
        feature_train.extend(outputs.cpu().numpy())
        labels_train.extend(labels.numpy())
        

In [20]:
print(labels_train)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18,

In [18]:
mat = np.array(feature_train)

In [21]:
print(len(mat), len(labels_train))

4016 4016


TypeError: Field elements must be 2- or 3-tuples, got '0'

In [23]:
np.save("featureEncoderLabledTrainingSet80.npy",mat)

In [20]:
test = ImageDataset(dataset=datasets.TEST,network_input_size=256,cuda = cuda)
test = DataLoader(test,128,False)

In [21]:
feature_test = []
encoder.eval()
with torch.no_grad():
    for inputs,_ in test:
        inputs = inputs.to(device)
        outputs = encoder(inputs)
        outputs = outputs.view(outputs.size(0),-1)  # Flatte
        feature_test.extend(outputs.cpu().numpy())
        

In [24]:
mat = np.array(feature_test)

In [25]:
print(mat)

[[0.43762147 0.51468194 0.44213897 ... 2.150414   0.7502871  1.0762559 ]
 [0.15512429 0.49725395 0.48254853 ... 0.69004756 0.6835057  0.89952165]
 [0.5630395  0.5237941  0.6279851  ... 1.3228151  0.7432212  0.8838805 ]
 ...
 [0.9894059  0.572808   0.68483895 ... 0.36716112 0.88989836 0.91502374]
 [0.35144174 0.27764052 0.22632214 ... 0.9308243  0.7506467  0.7002033 ]
 [0.6189376  0.76384467 0.3310122  ... 0.7337756  0.7471715  0.40838045]]


In [26]:
np.save("featureEncoderTestSet.npy",mat)

In [24]:
feature_val = []
labels_val = []
encoder.eval()
with torch.no_grad():
    for inputs,labels in validation:
        inputs = inputs.to(device)
        outputs = encoder(inputs)
        outputs = outputs.view(outputs.size(0),-1)  # Flatte
        feature_val.extend(outputs.cpu().numpy())
        labels_val.extend(labels.numpy())

In [25]:
mat = np.array(feature_val)

In [26]:
np.save("featureEncoderLabledValidationSet20.npy",mat)